# Exploring the Orderbook

In [1]:
from datetime import datetime, timedelta

import os
os.chdir("./")  # path to the base directory of RL4MM
from RL4MM.orderbook.create_order import create_order, OrderDict
from RL4MM.orderbook.models import Order, Orderbook, LimitOrder, MarketOrder, Cancellation, Deletion
from RL4MM.orderbook.Exchange import Exchange
from RL4MM.orderbook.visualise_orderbook import *

# Jedi not working
%config Completer.use_jedi = False

In [2]:
ticker="FAKE"
start = datetime(2019,1,2,10,35,45)
delta = timedelta(microseconds = 100000)

## To manage an Orderbook, create an ```Exchange```

each exchange automatically initialises an ```Orderbook``` to keep track of.

In [3]:
exchange = Exchange(ticker)
orderbook = exchange.orderbook

## Make some tools to create orders of different types

In [21]:
def get_limit_order(timestamp:datetime, price:float, volume:int, direction:str, internal_id: int):
    return LimitOrder(timestamp = timestamp, price = price, volume=volume, direction=direction, ticker = "MSFT,=", internal_id=internal_id, external_id=None, is_external= False)
def get_market_order(timestamp:datetime, volume:int, direction:str, internal_id: int=None):
    return MarketOrder(timestamp = timestamp, volume=volume, direction=direction, ticker = "MSFT,=", internal_id=internal_id, external_id=None, is_external= False)
def get_cancellation(timestamp:datetime, price: float, volume:int, direction:str, internal_id:int):
    return Cancellation(timestamp = timestamp, price = price, volume=volume, direction=direction, ticker = "MSFT,=",  internal_id = internal_id, external_id=None, is_external= False)
def get_deletion(timestamp:datetime, price: float, direction:str, internal_id):
    return Deletion(timestamp = timestamp, price = price, volume = None, direction=direction, ticker = "MSFT", internal_id = internal_id, external_id=None, is_external= False)

## Submitting limit orders

In [22]:
limit_bid_price_vol_tuples = ((99.8,10), (99.85, 50), (99.87, 100), (99.9, 80), (99.95, 100), (99.98, 200))
limit_ask_price_vol_tuples = ((100.1,20), (100.05, 30), (100.04, 60), (100.02, 80), (100.01, 150))
internal_id = 1
ts = start
limit_orders = list()
for price, vol in limit_bid_price_vol_tuples:
    limit_orders.append(get_limit_order(ts, price, vol, "bid", internal_id))
    internal_id +=1
    ts +=delta
for price, vol in limit_ask_price_vol_tuples:
    limit_orders.append(get_limit_order(ts, price, vol, "ask", internal_id))
    internal_id +=1
    ts +=delta       

In [23]:
for order in limit_orders:
    exchange.process_order(order)
visualise_orderbook(exchange.orderbook)

## Market orders

In [24]:
market_order = get_market_order(direction="ask", volume=150, timestamp=ts)
ts += delta
exchange.process_order(market_order)
visualise_orderbook(exchange.orderbook)

### Trying to place a market order that takes all the volume from the book, and more

Note that perhaps this is not the desired behaviour and we should let the market order take all the liquidity (as we currently do) but then just ignore the rest. To discuss.

In [8]:
market_order = get_market_order(direction="ask", volume=700, timestamp=ts)
ts += delta

exchange.process_order(market_order);

EmptyOrderbookError: Trying take liquidity from empty bid side of the book.

## Cancellations and Deletions

In [12]:
# reset orderbook
exchange.orderbook = exchange.get_empty_orderbook()
for order in limit_orders:
    exchange.process_order(order)

In [13]:
# We cancel half of the bid order at level 99.9
order = exchange.orderbook["bid"][99.9][0]
cancellation = get_cancellation(timestamp=ts, price = order.price, direction="bid", internal_id=order.internal_id, volume=order.volume/2)
ts += delta

In [14]:
exchange.process_order(cancellation)

In [15]:
visualise_orderbook(exchange.orderbook)

In [16]:
# Delete best order
order = exchange.orderbook["bid"][99.98][0]
deletion = get_deletion(timestamp=ts, price = order.price, direction="bid", internal_id=order.internal_id)
ts += delta

In [17]:
exchange.process_order(deletion)

In [18]:
visualise_orderbook(exchange.orderbook)

## Crossing the spread with a limit order

In [19]:
# reset orderbook
exchange.orderbook = exchange.get_empty_orderbook()
for order in limit_orders:
    exchange.process_order(order)
visualise_orderbook(exchange.orderbook)

In [20]:
limit_cross = get_limit_order(start, 100.03, 300, "bid", internal_id)
internal_id +=1
exchange.process_order(limit_cross)
visualise_orderbook(exchange.orderbook)